In [4]:
%pip install ragas

/home/nati/miniconda3/envs/redash/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=22294) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


  Using cached ragas-0.0.22-py3-none-any.whl.metadata (4.6 kB)
  Using cached pysbd-0.3.4-py3-none-any.whl (71 kB)
Using cached ragas-0.0.22-py3-none-any.whl (52 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 654.2 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings

ModuleNotFoundError: No module named 'langchain.embeddings.azure_openai'

In [3]:
import requests
import os
import weaviate
from weaviate.embedded import EmbeddedOptions
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter  
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

from dotenv import load_dotenv,find_dotenv

ImportError: cannot import name 'AzureOpenAIEmbeddings' from 'langchain.embeddings' (/home/nati/miniconda3/envs/redash/lib/python3.12/site-packages/langchain/embeddings/__init__.py)

In [5]:
# Data loader
def chunk_loader(file_path= '../RAG/prompts/context.txt'):
    loader = TextLoader(file_path)
    documents = loader.load()

    # Chunk the data
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(documents)
    return chunks


def create_retriever(chunks):
  # Load OpenAI API key from .env file
  load_dotenv(find_dotenv())

  # Setup vector database
  client = weaviate.Client(
    embedded_options = EmbeddedOptions()
  )

  # Populate vector database
  vectorstore = Weaviate.from_documents(
      client = client,    
      documents = chunks,
      embedding = OpenAIEmbeddings(),
      by_text = False
  )

  # Define vectorstore as retriever to enable semantic search
  retriever = vectorstore.as_retriever()
  return retriever


def file_reader(path: str, ) -> str:
    fname = os.path.join(path)
    with open(fname, 'r') as f:
        system_message = f.read()
    return system_message


def test_prompts():
    prompts = file_reader("../prompts/automatically-generated-prompts.txt")
    chunks =  chunk_loader()
    retriever = create_retriever(chunks)

    # Define LLM
    llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)

    final_prompts = []
    prompt_accuracy = []
    for prompt in prompts:
       final_prompts.append(ChatPromptTemplate.from_template(prompt))

    # prompt = ChatPromptTemplate.from_template(template)

    for prompt in final_prompts:
        # Setup RAG pipeline
        rag_chain = (
            {"context": retriever,  "question": RunnablePassthrough()} 
            | prompt 
            | llm
            | StrOutputParser() 
        )

        test_cases = file_reader("../prompts/automatically-generated-test-prompts.txt")

        questions = []
        ground_truths = []
        for test_case in test_cases:
            questions.append(test_case["user"])
            ground_truths.append(test_case["assistant"])

        answers = []
        contexts = []

        # Inference
        for query in questions:
            answers.append(rag_chain.invoke(query))
            contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

        # To dict
        data = {
            "question": questions, # list 
            "answer": answers, # list
            "contexts": contexts, # list list
            "ground_truths": ground_truths # list Lists
        }

        # Convert dict to dataset
        dataset = Dataset.from_dict(data)

        result = evaluate(
            dataset = dataset, 
            metrics=[
                context_precision,
                context_recall,
                faithfulness,
                answer_relevancy,
            ],
        )

        print(f"Test Values for {prompt}")
        print(result)


    return {final_prompts[i]:prompt_accuracy[i] for i in range(len(final_prompts))}

In [7]:
test_prompts()

FileNotFoundError: [Errno 2] No such file or directory: '../prompts/automatically-generated-prompts.txt'